<a href="https://colab.research.google.com/github/santiboub/DLDSProject/blob/main/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>